In [250]:
import requests
import pandas as pd
from bcb import sgs, Expectativas
from natsort import natsort_key
import numpy as np


In [251]:
# função para fazer a formatação das DataFrames

def formata_df(df: pd.DataFrame):
    
    # Colocando Reuniao como header
    df_pivot = df.pivot_table(index='Data', columns='Reuniao', values='Mediana')

    # Formatando ordem da Reuniao
    df_pivot = df_pivot.sort_index(axis=1)
    return df_pivot.applymap(lambda x: '{:.4f}'.format(x) if not pd.isnull(x) else np.nan)

In [252]:
em = Expectativas()
# em.describe()

ep_selic = em.get_endpoint('ExpectativasMercadoSelic')
ep_expec_anual = em.get_endpoint('ExpectativasMercadoAnuais')

In [253]:
# Selic mensal por reuniões

selic_mensal = ep_selic.query().filter(ep_selic.Data >= '01/01/2023').select(ep_selic.Data, ep_selic.Reuniao, ep_selic.Mediana).collect()
selic_mensal_pivot = formata_df(selic_mensal)


# 
idx = (selic_mensal_pivot.columns.to_series().str.split('/', expand=True)
         .sort_values(by=[1, 0], key=natsort_key).index
      )

selic_mensal_reuniao = selic_mensal_pivot[idx]
# selic_mensal_reuniao

In [254]:
# Selic anual

selic_anual = ep_expec_anual.query().filter(ep_expec_anual.Indicador == 'Selic', ep_expec_anual.Data >= '01/01/2023').select(ep_expec_anual.Indicador, ep_expec_anual.Data, ep_expec_anual.DataReferencia, ep_expec_anual.Mediana).collect() 

selic_anual['DataReferencia'] = pd.to_datetime(selic_anual['DataReferencia'], format='%Y')
selic_anual_pivot = selic_anual.pivot_table(index='Data', columns='DataReferencia', values='Mediana')    

selic_anual_pivot = selic_anual_pivot.sort_index(axis=1)
selic_anual_pivot.columns = selic_anual_pivot.columns.strftime('%Y')

# selic_anual_pivot


In [255]:
with pd.ExcelWriter(r'SEU_CAMINHO') as writer:
   selic_mensal_reuniao.to_excel(writer, sheet_name='Mediana mensal')
   selic_anual_pivot.to_excel(writer, sheet_name='Mediana anual', index=False)